In [11]:
import gym
from collections import deque
from IPython.display import clear_output
import time

init_state = 328
epochs = 0
rewards_record = 0
penalities_num = 0

env = gym.make('Taxi-v3').env
state = env.reset(seed = 41)

#print frames as animation
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(0.1)
        
        
# BFS node:
class BFSNode:
    def __init__(self, frame=None, state=None, action=None, reward=None , Parent_node = None):
        self.frame = frame
        self.state = state
        self.action = action
        self.reward = reward
        self.Parent_node = Parent_node
    

#given a current node explores the next states and returns a list of nodes.
def getNeigbours(node):
    env.unwrapped.s = node.state #might be a mistake
    neighbours = []
    for action in range(6):
        new_state, reward, done, info = env.step(action) # Take action i
        new_node = BFSNode(env.render(mode='ansi'),new_state,action,reward,node)
        neighbours.append(new_node)
        env.unwrapped.s = node.state #might be a mistake
    return neighbours



#returns a list of nodes that represents the shortest path from src_node to dst_node 
def findPathList(src_node,dst_node):
    parent = dst_node.Parent_node
    path_list = []
    reward = 0
    #path_list.append({'frame' : dst_node.frame ,'state' : dst_node.state,'action' : dst_node.action,'reward' : dst_node.reward})
    current_node = dst_node
    while parent != src_node.Parent_node:
        reward += current_node.reward
        path_list.append({'frame' : current_node.frame ,'state' : current_node.state,'action' : current_node.action,'reward' : current_node.reward})
        current_node = parent
        parent = current_node.Parent_node
    path_list.reverse()
    print(type(path_list))
    return path_list,reward

def compareTaxiAndDst(init_raw,init_col,dst):
    #print("compare is here!")
    if dst == 0: #red
        if init_raw == 0 and init_col == 0:
            return True
        return False
    elif dst == 1: # green
        if init_raw == 0 and init_col == 4:
            return True
        return False
    elif dst == 2: # yellow
        if init_raw == 4 and init_col == 0:
            return True
        return False
    elif dst == 3: #blue
        if init_raw == 4 and init_col == 3:
            return True
        return False
    return False #should't get here

def compareTaxiAndPass(raw,col,pass_loc):
    return compareTaxiAndDst(raw,col,pass_loc)

#returns true if state is in the list
def findNode(e_node,node_list):
    for node in node_list:
        if node.state == e_node.state and node.action == e_node.action and node.frame == e_node.frame and node.reward == e_node.reward:
            return True
    return False

######################BFS

# use the algorithm learnt in class
# should return (frames,reward)
def bfs(first_state):
    # add implementation of bfs-G
    dq_open = deque([]) #represents a queue of the next state that might be visited.
    dq_close = deque([])  #contains the visited states.
    frames = []
    reward = 0
    picked_up = False
    init_node = BFSNode(env.render(), first_state)
    init_raw,init_col,init_pass_loc,init_d= env.decode(init_node.state)

    if init_pass_loc:
        picked_up = True
    env.unwrapped.s = first_state #setting the initial state
    dq_open.append(init_node)
    while len(dq_open) > 0 :
        curr = dq_open.popleft()
        dq_close.append(curr)
        curr_raw,curr_col,curr_pass_loc,curr_d= env.decode(curr.state)
        
        #did we reach the pass loc and picked him up in a legal way?
        if picked_up == False:
            if compareTaxiAndPass(curr_raw,curr_col,init_pass_loc) and curr.action == 4: 
                picked_up = True
                
                
        #if we already picked up the pass
        #we reached the dst node and dropped off
        if picked_up == True:
            if compareTaxiAndDst(curr_raw,curr_col,init_d) and curr.action == 5: 
                print(3)
                path, reward = findPathList(init_node,curr)
                return path, reward
                #dq_open.append(e)
                
        neighbours_nodes = getNeigbours(curr)
        for e in neighbours_nodes: #assuming neighbours are valid
            #print(e.action)
            if not (findNode(e,dq_open) or findNode(e,dq_close)): #TODO check the place of not
                new_node = (e.frame,e.state,e.action,e.reward,curr)
                dq_open.append(e)
    return [],0


############DFS

def findNodeDFS(e_node,node_list):
    for node in node_list:
        if node.state == e_node.state:
            if e_node.action != 4 or  e_node.action != 5:
                return True
            if node.action == e_node.action:
                return True
    return False
    
    
def dfs_aux(dq_visited,dq_stack,picked_up,init_node):
    init_raw,init_col,init_pass_loc,init_d= env.decode(init_node.state)
    if len(dq_stack) == 0: #shouldn't get here
        return [],0
    curr = dq_stack.popleft()
    dq_visited.appendleft(curr)
    curr_raw,curr_col,curr_pass_loc,curr_d= env.decode(curr.state)
        
    #did we reach the pass loc and picked him up in a legal way?
    if picked_up == False:
        if compareTaxiAndPass(curr_raw,curr_col,init_pass_loc) and curr.action == 4: 
            picked_up = True
                
                
    #if we already picked up the pass
    #we reached the dst node and dropped off
    if picked_up == True:
        if compareTaxiAndDst(curr_raw,curr_col,init_d) and curr.action == 5: 
            path, reward = findPathList(init_node,curr)
            return path, reward 
                
    neighbours_nodes = getNeigbours(curr)
    tmp = len(neighbours_nodes)
    for i in range(tmp):
        if findNodeDFS(neighbours_nodes[tmp-i-1],dq_visited):
            continue
        dq_stack.appendleft(neighbours_nodes[tmp-i-1])
    for e in neighbours_nodes:
        path, reward = dfs_aux(dq_visited,dq_stack,picked_up,init_node)
        return path, reward
    return [],0
    
#we need to wrap the function
def dfs(first_state):
    # add implementation
    dq_visited = deque([])  #contains the visited nodes.
    dq_stack = deque([]) #contains the explored nodes that we can visit.
    picked_up = False
    init_node = BFSNode(env.render(), first_state)
    dq_stack.appendleft(init_node)
    init_raw,init_col,init_pass_loc,init_d= env.decode(init_node.state)
    #the passenger starts standing on the goal point
    if init_pass_loc == init_d:
        return [],20
    #the passenger is in the taxi
    if init_pass_loc == 4:
        picked_up = True
    #if compareTaxiAndPass(init_raw,init_col,init_pass_loc):
    #    picked_up = False
    path, reward = dfs_aux(dq_visited,dq_stack,picked_up,init_node)
    return path, reward


#env.unwrapped.s = 328
#frames,reward = dfs(env.s)
#print_frames(frames)
#print(reward)

#############ID_DFS


# ID_DFS node:
class ID_DFSNode:
    def __init__(self, frame=None, state=None, action=None, reward=None , Parent_node = None, depth=0):
        self.frame = frame
        self.state = state
        self.action = action
        self.reward = reward
        self.Parent_node = Parent_node
        self.depth = depth

#given a current node explores the next states and returns a list of nodes.
def ID_DFSgetNeigbours(node):
    env.unwrapped.s = node.state #might be a mistake
    neighbours = []
    for action in range(6):
        new_state, reward, done, info = env.step(action) # Take action i
        new_node = ID_DFSNode(env.render(mode='ansi'),new_state,action,reward,node,node.depth+1)
        neighbours.append(new_node)
        env.unwrapped.s = node.state #might be a mistake
    return neighbours


def dfs_L_aux(max_depth,dq_visited,dq_stack,picked_up,init_node):
    init_raw,init_col,init_pass_loc,init_d= env.decode(init_node.state)
    if len(dq_stack) == 0: #shouldn't get here
        return 'fail',[],0
    curr = dq_stack.popleft()
    dq_visited.appendleft(curr)
    curr_raw,curr_col,curr_pass_loc,curr_d= env.decode(curr.state)
        
    #did we reach the pass loc and picked him up in a legal way?
    if picked_up == False:
        if compareTaxiAndPass(curr_raw,curr_col,init_pass_loc) and curr.action == 4: 
            picked_up = True
                
                
    #if we already picked up the pass
    #we reached the dst node and dropped off
    if picked_up == True:
        if compareTaxiAndDst(curr_raw,curr_col,init_d) and curr.action == 5: 
            path, reward = findPathList(init_node,curr)
            return 'success',path, reward 
                
    neighbours_nodes = ID_DFSgetNeigbours(curr)
    tmp = len(neighbours_nodes)
    for i in range(tmp):
        if findNodeDFS(neighbours_nodes[tmp-i-1],dq_visited) or neighbours_nodes[tmp-i-1].depth > max_depth:
            continue
        dq_stack.appendleft(neighbours_nodes[tmp-i-1])
    for e in neighbours_nodes:
        status,path, reward = dfs_L_aux(max_depth,dq_visited,dq_stack,picked_up,init_node)
        return status,path, reward
    return 'fail',[],0


def dfs_L(first_state, max_depth):
    # add implementation
    L = 0
    dq_visited = deque([])  #contains the visited nodes.
    dq_stack = deque([]) #contains the explored nodes that we can visit.
    picked_up = False
    init_node = ID_DFSNode(env.render(), first_state)
    dq_stack.appendleft(init_node)
    init_raw,init_col,init_pass_loc,init_d= env.decode(init_node.state)
    #the passenger starts standing on the goal point
    if init_pass_loc == init_d:
        return 'success',[],20
    #the passenger is in the taxi
    if init_pass_loc == 4:
        picked_up = True
    status,Path,reward = dfs_L_aux(max_depth,dq_visited,dq_stack,picked_up,init_node)
    if status != 'fail':
        return 'success',Path,reward
    
    return 'fail',[],0       # remove me


# use the algorithm learnt in class
# think about how to use dfs() foor your solution
# should return (frames,reward)

def id_dfs(first_state, initial_depth):
    L = 0
    status = 'fail'
    while L <= initial_depth:
        status,Path,reward = dfs_L(first_state, L)
        if status != 'fail':
            return Path,reward
        L += 1
    return [],0


# driver code - for your use only, do not submit these lines!
#env.s = 328
#initial_depth = 500 # random depth
#frames,reward = id_dfs(env.s, initial_depth)
#print_frames(frames)
#print(reward)




###############WA* 

def null_h(state):#khalsa
    return 0 


def MD_calc(s_col,s_raw,d_col,d_raw):
    col_dist = 0
    raw_dist = 0
    if d_col >= s_col:
        col_dist = d_col - s_col
    else:
        col_dist = s_col - d_col
    if d_raw >= s_raw :
        raw_dist = d_raw - s_raw
    else:
        raw_dist = s_raw - d_raw
    return raw_dist + col_dist


def DstToCoordinates(dst):#convers pass dst to raw and col
    raw,col = 0,0 
    if dst == 0: #red
        raw = 0
        col = 0
        return raw,col
    if dst == 1: # green
        raw = 0
        col = 4
        return raw,col
    if dst == 2: # yellow
        raw = 4
        col = 0
        return raw,col
    if dst == 3: #blue
        raw = 4
        col = 3
        return raw,col
    return 0,0
    
    
    
def chosen_h(state): # implement it as admissible_h too
    #we choose the second MD function
    # add implementation
    init_raw,init_col,init_pass_loc,init_dst= env.decode(state)
    pass_raw,pass_col = DstToCoordinates(init_pass_loc)
    MD_pick = MD_calc(init_col,init_raw,pass_col,pass_raw)
    dst_raw,dst_col = DstToCoordinates(init_pass_loc)
    MD_drop = MD_calc(pass_col,pass_raw,dst_col,dst_raw)
    return MD_pick + MD_drop


def admissible_h(state):#exactly like chosen_h for testing
    # add implementation
    init_raw,init_col,init_pass_loc,init_dst= env.decode(state)
    pass_raw,pass_col = DstToCoordinates(init_pass_loc)
    MD_pick = MD_calc(init_col,init_raw,pass_col,pass_raw)
    dst_raw,dst_col = DstToCoordinates(init_pass_loc)
    MD_drop = MD_calc(pass_col,pass_raw,dst_col,dst_raw)
    return MD_pick + MD_drop

def non_admissible_h(state):
    init_raw,init_col,init_pass_loc,init_dst= env.decode(state)
    picked_up = False
    if init_pass_loc == 4:
        picked_up = True
    if picked_up == False:
        if compareTaxiAndPass(init_raw,init_col,init_pass_loc): 
            return 1
    if picked_up == True:
        if compareTaxiAndDst(init_raw,init_col,init_dst):
            return 1
    return 0

class WA_Node:
    def __init__(self,frame=None, state=None, action=None, reward=None , Parent_node = None, g = None, h = None):
        self.frame = frame
        self.state = state
        self.action = action
        self.reward = reward
        self.Parent_node = Parent_node
        self.g = g
        self.h = h


def WA_getNeigbours(node):
    env.unwrapped.s = node.state #might be a mistake
    neighbours = []
    for action in range(6):
        new_state, reward, done, info = env.step(action) # Take action i
        new_node = WA_Node(env.render(mode='ansi'),new_state,action,reward,node,None,None)
        neighbours.append(new_node)
        env.unwrapped.s = node.state #might be a mistake
    return neighbours
#
def update_node(old_node,new_node):
    old_node.frame = new_node.frame
    old_node.state = new_node.state
    old_node.action = new_node.action
    old_node.reward = new_node.reward
    old_node.Parent_node = new_node.Parent_node
    old_node.g = new_node.g
    old_node.h = new_node.h
    
    
def find_WA_Node(curr,node_list):
    for node in node_list:
        if node.state == curr.state:
            return True
    return False
 
def getNode(curr,node_list):
    for node in node_list:
        if node.state == curr.state:
            return node
    return None ## shoudln't get here
    
    
def get_min(node_list,weight):
    min_node = node_list[0]  
    min_f = (weight)*min_node.h + (1-weight)*min_node.g #TODO
    index = 0
    min_index = index
    for node in node_list:
        f = (weight)*node.h + (1-weight)*node.g 
        index = index + 1
        if f < min_f:
            min_node = node
            min_f = f
            min_index = index - 1
            
    del node_list[min_index]
    return min_node

def remove_node(node_list,node):
    index = 0
    for n in node_list:
        if n == node:
            break
        index = index + 1
            
    del node_list[index]
    return 

## calculate the distanse from init
def calculate_g(init_state,curr):  
    init_raw,init_col,init_pass_loc,init_d = env.decode(init_state)
    cur_raw,cur_col,cur_pass_loc,cur_d = env.decode(curr)
    return MD_calc(init_col,init_raw,cur_col,cur_raw)


def weighted_a_star(first_state,weight,heuristic_function):
    WA_open = deque([])
    WA_close = deque([]) 
    frames = []
    reward = 0
    picked_up = False
    
    ####setting the initial state
    init_node = WA_Node(env.render(), first_state)
    init_node.g = 0
    init_node.h = heuristic_function(init_node.state)
    init_raw,init_col,init_pass_loc,init_d = env.decode(init_node.state)
    env.unwrapped.s = first_state
    WA_open.append(init_node)
    
    while len(WA_open) > 0 :
        curr = get_min(WA_open,weight)  ##should return the min of f 
        #print(curr.state)
        WA_close.append(curr)
        curr_raw,curr_col,curr_pass_loc,curr_d = env.decode(curr.state)
        
        #did we reach the pass loc and picked him up in a legal way?
        if picked_up == False:
            if compareTaxiAndPass(curr_raw,curr_col,init_pass_loc) and curr.action == 4: 
                picked_up = True
                ###should clear open queue and close queue
                
        #if we already picked up the pass check if we reached the dst node to drop off
        if picked_up == True:
            if compareTaxiAndDst(curr_raw,curr_col,init_d) and curr.action == 5: 
                path, reward = findPathList(init_node,curr) ## not sure if we can use findPathList
                return path, reward
            
        
        neighbours_nodes = WA_getNeigbours(curr)
        for e in neighbours_nodes: 
            e.g = calculate_g(init_node.state,curr.state)
            e.h = heuristic_function(e.state) 
            #new_node = (e.frame,e.state,e.action,e.reward,curr,e.g, e.h)
            #if not(find_WA_Node(e,WA_open) and find_WA_Node(e,WA_close)):   ##add new node to open in case if it wasn't before 
                
                
            if find_WA_Node(e,WA_close): ## NEW it could be a shorter road so we should upadte 
                old_e = getNode(e,WA_close)
                old_f = weight*old_e.h + (1-weight)*old_e.g
                new_f = weight*e.h + (1-weight)*e.g
                if(old_f > new_f):
                    ##should remove old_e from close and add e to open  
                    WA_open.append(e)
                    remove_node(WA_close,old_e)
                    
            elif find_WA_Node(e,WA_open): ## if it was in open compare weight and keep the smlallest
                old_e = getNode(e,WA_open)
                old_f = weight*old_e.h + (1-weight)*old_e.g
                new_f = weight*e.h + (1-weight)*e.g
                if(old_f > new_f):
                    ##should update old_e 
                    update_node(old_e,e)
            else:         
                WA_open.append(e)
    return [],0




# driver code - for your use only, do not submit these lines!
#env.unwrapped.s = 328
#weight = 1   # you will need to change this during the excersize 
#frames,reward = weighted_a_star(env.s,weight,chosen_h)
#print_frames(frames)
#print(reward)
#frames,reward = weighted_a_star(env.s,weight,non_admissible_h)
#print_frames(frames)
#print(reward)




###########AStar 

def WAE_get_min(node_list,weight,min_val):
    min_node = node_list[0]  
    index = 0
    min_index = index
    for node in node_list:
        f = (weight)*node.h + (1-weight)*node.g 
        index = index + 1
        if f <= min_val:
            min_node = node
            min_index = index - 1
            break
    del node_list[min_index]
    return min_node

def update_min(node_list,weight):
    min_node = node_list[0]  
    min_f = (weight)*min_node.h + (1-weight)*min_node.g
    for node in node_list:
        f = (weight)*node.h + (1-weight)*node.g 
        if f < min_f:
            min_f = f
    return min_f

def a_star_epsilon(first_state,weight,heuristic_function, epsilon):
    # add implementation
    WA_open = deque([])
    WA_close = deque([]) 
    frames = []
    reward = 0
    picked_up = False
    
    ####setting the initial state
    init_node = WA_Node(env.render(), first_state)
    init_node.g = 0
    init_node.h = heuristic_function(init_node.state)
    init_raw,init_col,init_pass_loc,init_d = env.decode(init_node.state)
    env.unwrapped.s = first_state
    WA_open.append(init_node)
    min_f_val = (1-weight)*init_node.g + (weight)*init_node.h

    while len(WA_open) > 0 :
        min_val = (1+epsilon)*min_f_val
        curr = WAE_get_min(WA_open,weight,min_val)  ##should return the first node with f <= min_val
        if len(WA_open) > 0:
            min_f_val = update_min(WA_open,weight)
        
        #print(curr.state)
        WA_close.append(curr)
        curr_raw,curr_col,curr_pass_loc,curr_d = env.decode(curr.state)
        
        #did we reach the pass loc and picked him up in a legal way?
        if picked_up == False:
            if compareTaxiAndPass(curr_raw,curr_col,init_pass_loc) and curr.action == 4: 
                picked_up = True
                ###should clear open queue and close queue
                
        #if we already picked up the pass check if we reached the dst node to drop off
        if picked_up == True:
            if compareTaxiAndDst(curr_raw,curr_col,init_d) and curr.action == 5: 
                path, reward = findPathList(init_node,curr) ## not sure if we can use findPathList
                return path, reward
            
        
        neighbours_nodes = WA_getNeigbours(curr)
        for e in neighbours_nodes: 
            e.g = calculate_g(init_node.state,curr.state)
            e.h = heuristic_function(e.state) 
            #new_node = (e.frame,e.state,e.action,e.reward,curr,e.g, e.h)
            #if not(find_WA_Node(e,WA_open) and find_WA_Node(e,WA_close)):   ##add new node to open in case if it wasn't before 
                
                
            if find_WA_Node(e,WA_close): ## NEW it could be a shorter road so we should upadte 
                old_e = getNode(e,WA_close)
                old_f = weight*old_e.h + (1-weight)*old_e.g
                new_f = weight*e.h + (1-weight)*e.g
                if(old_f > new_f):
                    ##should remove old_e from close and add e to open  
                    WA_open.append(e)
                    if new_f < min_f_val:
                        min_f_val = new_f
                    remove_node(WA_close,old_e)
                    
            elif find_WA_Node(e,WA_open): ## if it was in open compare weight and keep the smlallest
                old_e = getNode(e,WA_open)
                old_f = weight*old_e.h + (1-weight)*old_e.g
                new_f = weight*e.h + (1-weight)*e.g
                if(old_f > new_f):
                    ##should update old_e 
                    update_node(old_e,e)
            else:         
                WA_open.append(e)
                if weight*e.h + (1-weight)*e.g < min_f_val:
                    min_f_val = weight*e.h + (1-weight)*e.g
    return [],0


# driver code - for your use only, do not submit these lines!
#env.s = 328
#weight = 3   # you will need to change this during the excersize 
#frames,reward  = weighted_a_star(env.s,weight,admissible_h)
#print_frames(frames)
#print(reward)
#frames,reward = weighted_a_star(env.s,weight,non_admissible_h)
#print_frames(frames)
#print(reward)





+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 18
State: 0
Action: 5
Reward: 20
3
